
#### OBJETIVO: Realizar el promedio de las mediciones de AERONET.
Para un intervalo de tiempo determinado centrado en el paso del satelite, con el fin de compararlo con el promedio de las recuperaciones de MAIAC.


In [1]:
#Librerias
import pandas as pd
import numpy as np
from datetime import timedelta


In [2]:
def time_correlation(path_aeronet, path_maiac, time_buffer, formato_fecha):
    """
    Promedia AOD AERONET dentro de una ventana temporal centrada en el paso del satelite
    """

    # --- AERONET ---
    data_aeronet = pd.read_csv(path_aeronet, na_values="NA")
    data_aeronet["date"] = pd.to_datetime(
        data_aeronet["date"],
        format=formato_fecha,
        utc=True
    )

    # --- MAIAC ---
    data_maiac = pd.read_csv(path_maiac, na_values="NA")
    data_maiac = data_maiac.dropna(subset=["AOD_055"])

    data_maiac["date"] = pd.to_datetime(
        data_maiac["date"].astype(str),
        format="%Y%j",
        utc=True
    )

    data_maiac["hour"] = pd.to_datetime(
        data_maiac["timestamp"].astype(str),
        format="%Y%j%H%M",
        utc=True
    )

    AOD = []

    for i, row in data_maiac.iterrows():
        if i % 50 == 0:
            print(i)

        # Coincidencia dia
        aer_day = data_aeronet[
            (data_aeronet["date"].dt.year == row["date"].year) &
            (data_aeronet["date"].dt.month == row["date"].month) &
            (data_aeronet["date"].dt.day == row["date"].day)
        ]

        if aer_day.empty:
            continue

        # Ventana temporal
        delta = abs(aer_day["date"] - row["hour"])
        aer_window = aer_day[delta <= timedelta(minutes=time_buffer)]

        if aer_window.empty:
            continue

        # Estadisticos AERONET
        mean_aer = aer_window.iloc[:, 4].mean()
        med_aer = aer_window.iloc[:, 4].median()
        sd_aer = aer_window.iloc[:, 4].std()
        dim_aer = len(aer_window)

        df = {
            "Date_MODIS": row.iloc[1],
            "timestamp": row.iloc[15],
            "satellite": row.iloc[9],
            "AOD_470": row.iloc[10],
            "AOD_550_maiac": row.iloc[11],
            "uncert": row.iloc[12],
            "date_AERO": aer_window.iloc[0, 0][:10],
            "AOD_550_AER_mean": mean_aer,
            "AOD_550_AER_median": med_aer,
            "AOD_550_AER_sd": sd_aer,
            "AOD_550_AER_dim": dim_aer
        }

        AOD.append(df)

    return pd.DataFrame(AOD)


In [ ]:
buffer_time = 30
buffer_spatial = "1km"
city = "GT"
num_site = "2"
region = "USA"
formato_fecha = "%d/%m/%Y %H:%M"

data_maiac = f"D:/Josefina/paper_git/paper_maiac/datasets/V03/maiac/{region}_C61/{city}/prueba_{buffer_spatial}_{city}_C61_tot_V03.csv"
data_aeronet = f"D:/Josefina/paper_git/paper_maiac/datasets/V03/aeronet/datasets_interp_s_L02/{region}/{num_site}_{city}_2015-2024_interp-s_V03_L2.csv"


combinate = time_correlation(
    path_aeronet=data_aeronet,
    path_maiac=data_maiac,
    time_buffer=buffer_time,
    formato_fecha=formato_fecha
)

# combinate.to_csv(
#     f"D:/Josefina/paper_git/paper_maiac/datasets/V03/processed/merge_AER-MAIAC/{region}_C61/tot/{buffer_spatial}/{num_site}_{city}-{buffer_spatial}-MAIAC-{buffer_time}-AER_C61.csv",
#     index=False
# )
